In [2]:
from tensorflow.keras.models import load_model

# kalau model TIDAK memakai tf_hub.KerasLayer:

infer_model = load_model("model_lstm_1.keras")  # compile=False juga boleh untuk inference murni

# load mapping kelas
import json
with open("class_names.json") as f:
    idx_to_star = {int(k): v for k, v in json.load(f).items()}


In [3]:
import tensorflow as tf
import numpy as np

sample_texts = [
    "The food was cold and the staff were rude. Terrible experience.",
    "Service was slow and my order was wrong, but the place was clean.",
    "Food was okay, nothing special but not bad either.",
    "Nice place, friendly staff, and good fries. I enjoyed it.",
    "Everything was perfect! Delicious food, fast service, and kind staff."
]

x_pred = tf.constant(sample_texts, dtype=tf.string)   # <- ini kunci anti error dtype
probs  = infer_model.predict(x_pred)                  # shape: (N, 5)
pred_i = probs.argmax(axis=1)                         # 0..4

for txt, i in zip(sample_texts, pred_i):
    print(f"{idx_to_star[i]}  →  {txt}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1 star  →  The food was cold and the staff were rude. Terrible experience.
3 stars  →  Service was slow and my order was wrong, but the place was clean.
4 stars  →  Food was okay, nothing special but not bad either.
4 stars  →  Nice place, friendly staff, and good fries. I enjoyed it.
5 stars  →  Everything was perfect! Delicious food, fast service, and kind staff.


In [4]:
import pandas as pd
pd.DataFrame(probs, columns=["1 star","2 stars","3 stars","4 stars","5 stars"])


,1 star,2 stars,3 stars,4 stars,5 stars
0,0.988787,0.004696,0.003032,0.001371,0.002113
1,0.005756,0.008870,0.954265,0.015890,0.015219
2,0.001301,0.008960,0.016784,0.947470,0.025485
3,0.000434,0.002193,0.016885,0.824092,0.156396
4,0.001013,0.000278,0.002693,0.016609,0.979408


In [5]:
def predict_rating(texts, model, idx_to_star):
    x = tf.constant(texts, dtype=tf.string)
    p = model.predict(x)
    top = p.argmax(axis=1)
    labels = [idx_to_star[i] for i in top]
    return labels, p

# contoh pakai:
labels, p = predict_rating(
    ["The food was cold and the staff were rude.",
    "Service was slow, but the burger was decent.",
    "Everything was perfect! Friendly staff and delicious food."],
    infer_model, idx_to_star
)
for t, lab in zip([
                    "The food was cold and the staff were rude.",
                    "Service was slow, but the burger was decent.",
                    "Everything was perfect! Friendly staff and delicious food."], labels):
    print(lab, "→", t)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 796ms/step
1 star → The food was cold and the staff were rude.
3 stars → Service was slow, but the burger was decent.
5 stars → Everything was perfect! Friendly staff and delicious food.
